# Luke Formulary Attempt
### v3.0

Let's start simple and see if we can determine which drugs are in a payer's formulary (reject code for formularies is 70)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('../data/dim_claims.csv')


In [3]:
print(df.shape)
list(df.columns)
df.head(5)

(1335576, 5)


,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved
0,1,417380,A,75.0,0
1,2,999001,A,NaN,1
2,3,417740,A,76.0,0
3,4,999001,A,NaN,1
4,5,417740,A,NaN,1


In [4]:
#X_train,X_test,y_train,y_test = train_test_split(X,y,
#                                                 test_size = .25,
#                                                 random_state = 614,
#                                                 shuffle = True,
#                                                 stratify = y)

#Start with 25% of the data set to keep the size managable and avoid biasing our results
df_train = df.sample(frac = .75)

#For each payer, why is the drug rejected?
payers = df_train['bin'].unique()
drugs = df_train['drug'].unique()
print(drugs)
print(payers)
print(type(payers))





['A' 'B' 'C']
[999001 417740 417614 417380]
<class 'numpy.ndarray'>


In [5]:
#cl = ['Payer']
#cl.extend(drugs)
#print(type(cl))

#print(cl)
df_rej= pd.DataFrame(columns=drugs)
df_rej.insert(loc=0,column='Payer',value=payers)

#den = df_train[(df_train['bin'] == payers[0])  & (df_train['drug']==drugs[0])].shape[0]
#rej = df_train[(df_train['bin'] == payers[0])  & (df_train['drug']==drugs[0]) & (df_train['pharmacy_claim_approved']==0)].shape[0]
#print(den)
#print(rej)
i = 0

#df_rej['Payer'] = payers

for payer in payers:
    for drug in drugs:
        den = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug)].shape[0]
        #print("Den = ",den)
        rej = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) & (df_train['pharmacy_claim_approved']==0)].shape[0]
        acc = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) & (df_train['pharmacy_claim_approved']==1)].shape[0]
        rejfrac = rej/den
     #   print("Payer ",payer,"rejects drug",drug," = ",rejfrac)
        df_rej.loc[df_rej['Payer'] == payer, [drug]] = rejfrac
        print("Total = ",acc,"+",rej," = ", acc+rej,", should equal", den)
        
df_rej.head()     


#df_train[(df_train['bin'] == 417740)  & (df_train['drug']=='B') & (df_train['pharmacy_claim_approved']==1)]


Total =  219871 + 24355  =  244226 , should equal 244226
Total =  111284 + 12229  =  123513 , should equal 123513
Total =  101497 + 11455  =  112952 , should equal 112952
Total =  73628 + 8036  =  81664 , should equal 81664
Total =  0 + 41207  =  41207 , should equal 41207
Total =  0 + 37521  =  37521 , should equal 37521
Total =  0 + 117403  =  117403 , should equal 117403
Total =  0 + 59061  =  59061 , should equal 59061
Total =  48626 + 5315  =  53941 , should equal 53941
Total =  0 + 66527  =  66527 , should equal 66527
Total =  29902 + 3276  =  33178 , should equal 33178
Total =  0 + 30489  =  30489 , should equal 30489


,Payer,A,B,C
0,999001,0.099723,0.09901,0.101415
1,417740,0.098403,1.00000,1.000000
2,417614,1.000000,1.00000,0.098534
3,417380,1.000000,0.09874,1.000000


So, using this simple method, we se that 

Payer 999001 has all three drugs in their formulary


Payer 417614 has only C in their formulary

Payer 417740 has only A in their formulary

Payer 417380 has has only B in their formlary

If this is correct, then we should see only 70 rejection codes for non-forumlary drugs and non-70 codes for formulary rejections

In [6]:
df_code= pd.DataFrame(columns=drugs)

df_code.insert(loc=0,column='Payer',value=0)
df_code.insert(loc=1,column='Acc/Code',value=0)


print("Codes in df_train are currently:",df_train['reject_code'].unique())

print("Reject codes with payer approval: ",
      df_train[df_train['pharmacy_claim_approved']==1]['reject_code'].unique())
print("Reject codes with payer denial: ",
      df_train[df_train['pharmacy_claim_approved']==0]['reject_code'].unique())
#So, all codes seem to be what they should be
df_train['reject_code'] = df_train['reject_code'].fillna(-1)

print("Reject codes with payer approval: ",
      df_train[df_train['pharmacy_claim_approved']==1]['reject_code'].unique())
print("Reject codes with payer denial: ",
      df_train[df_train['pharmacy_claim_approved']==0]['reject_code'].unique())
codes = df_train['reject_code'].unique()
print(codes)
#print(df_train.head(10))

rep = len(codes)
print(rep)

p = payers.repeat(rep)
print(p)

df_code['Payer'] = p
 
#pd.DataFrame([[payer for payer in payers],[1]], columns=['Payer','Acc/Code'])
for payer in payers:
    df_code.loc[df_code['Payer'] == payer,['Acc/Code']] = codes
    
df_code.head(20)

Codes in df_train are currently: [76. nan 75. 70.]
Reject codes with payer approval:  [nan]
Reject codes with payer denial:  [76. 75. 70.]
Reject codes with payer approval:  [-1.]
Reject codes with payer denial:  [76. 75. 70.]
[76. -1. 75. 70.]
4
[999001 999001 999001 999001 417740 417740 417740 417740 417614 417614
 417614 417614 417380 417380 417380 417380]


,Payer,Acc/Code,A,B,C
0,999001,76.0,NaN,NaN,NaN
1,999001,-1.0,NaN,NaN,NaN
2,999001,75.0,NaN,NaN,NaN
3,999001,70.0,NaN,NaN,NaN
4,417740,76.0,NaN,NaN,NaN
5,417740,-1.0,NaN,NaN,NaN
6,417740,75.0,NaN,NaN,NaN
7,417740,70.0,NaN,NaN,NaN
8,417614,76.0,NaN,NaN,NaN
9,417614,-1.0,NaN,NaN,NaN


In [7]:


#den = df_train[(df_train['bin'] == payers[0])  & (df_train['drug']==drugs[0])].shape[0]
#rej = df_train[(df_train['bin'] == payers[0])  & (df_train['drug']==drugs[0]) & (df_train['pharmacy_claim_approved']==0)].shape[0]
#print(den)
#print(rej)
i = 0

#df_code['Payer'] = payers

for payer in payers:
    for drug in drugs:
        for code in codes:
            den = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug)].shape[0]
        #print("Den = ",den)
            if(code>0):
                rej = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) 
                               & (df_train['pharmacy_claim_approved']==0) & (df_train['reject_code']==code)].shape[0]
            else:
                rej = 0    
            acc = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) & (df_train['pharmacy_claim_approved']==1)].shape[0]
            rejfrac = rej/den
            accfrac = acc/den
     #   print("Payer ",payer,"rejects drug",drug," = ",rejfrac)
            if(code>0):
                df_code.loc[(df_code['Payer'] == payer) & (df_code['Acc/Code']==code), [drug]] = rejfrac
            else:
                df_code.loc[(df_code['Payer'] == payer) & (df_code['Acc/Code']==code), [drug]] = accfrac
        #print("Total = ",acc,"+",rej," = ", acc+rej,", should equal", den)
        
df_code.head(25)   



,Payer,Acc/Code,A,B,C
0,999001,76.0,0.099723,0.09901,0.101415
1,999001,-1.0,0.900277,0.90099,0.898585
2,999001,75.0,0.000000,0.00000,0.000000
3,999001,70.0,0.000000,0.00000,0.000000
4,417740,76.0,0.098403,0.00000,0.000000
5,417740,-1.0,0.901597,0.00000,0.000000
6,417740,75.0,0.000000,0.00000,1.000000
7,417740,70.0,0.000000,1.00000,0.000000
8,417614,76.0,0.000000,0.00000,0.098534
9,417614,-1.0,0.000000,0.00000,0.901466


A more detailed view now emerges

999001: No drug is rejeced due to formulay or requires prior authorization. All druges are rejected due to code 76

417614: Drug A is not in formulary (always rejected, always due to code 70)
        Drug B is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        Drug C is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        
417740: Drug A is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        Drug B is not in formulary (always rejected, always due to code 70)
        Drug C is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        
417380: Drug A is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        Drug B is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        Drug C is not in formulary (always rejected, always due to code 70)

In [8]:
df_code.to_csv("formulary.csv",index=False)